<a href="https://colab.research.google.com/github/DataCrusade1999/expert-winner/blob/main/Sentiment_Analysis_With_Scikit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1561]:
import io
import pickle
import numpy as np
import pandas as pd
import datetime
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

**LOADING DATA**

In [1562]:
url='https://raw.githubusercontent.com/DataCrusade1999/Python/master/Data/Books_small_10000.csv'

In [1563]:
Books_small=pd.read_csv(url)
Books_small.head()

,reviewerID,asin,reviewerName,helpful/0,helpful/1,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1F2H80A1ZNN1N,B00GDM3NQC,Connie Correll,0,0,"I bought both boxed sets, books 1-5. Really a...",5,Can't stop reading!,1390435200,"01 23, 2014"
1,AI3DRTKCSK4KX,B00A5MREAM,Grandma,0,0,I enjoyed this short book. But it was way way ...,3,A leaf on the wind of all hallows,1399593600,"05 9, 2014"
2,A3KAKFHY9DAC8A,0446547573,"toobusyreading ""Inspired Kathy""",1,1,I love Nicholas Sparks. I&#8217;ve read everyt...,4,Great writing from Nicholas Sparks.,1404518400,"07 5, 2014"
3,ATYBCYD6BIXVL,0955809215,Chrissie,0,0,I really enjoyed this adventure and look forwa...,4,great,1389225600,"01 9, 2014"
4,A17K95SEU3J68U,0991500776,"Sirde ""artist761""",0,0,It was a decent read.. typical story line. Not...,3,It was a decent read.. typical story line ...,1404864000,"07 9, 2014"


**PERFORMING EDA**

**In the first case below, we say "give us the values of the rows with index from 0 to 999 (inclusive) and columns labeled from "reviewText" to "overall" (inclusive)". In the second case, we say "give us the values of all the rows in the columns 5 to 7" (as in a typical Python slice: the maximal value is not included).**

In [1564]:
Books_small.loc[0:,'reviewText':'overall']

,reviewText,overall
0,"I bought both boxed sets, books 1-5. Really a...",5
1,I enjoyed this short book. But it was way way ...,3
2,I love Nicholas Sparks. I&#8217;ve read everyt...,4
3,I really enjoyed this adventure and look forwa...,4
4,It was a decent read.. typical story line. Not...,3
...,...,...
9995,The whole series was great! Melody is a fanta...,5
9996,I didn't thing that much of this book. I am a...,3
9997,It is an emotional TRIP to the past with Trip ...,5
9998,This definitely got under my veins whereby I h...,5


In [1565]:
Books_small.iloc[0:,5:7]

,reviewText,overall
0,"I bought both boxed sets, books 1-5. Really a...",5
1,I enjoyed this short book. But it was way way ...,3
2,I love Nicholas Sparks. I&#8217;ve read everyt...,4
3,I really enjoyed this adventure and look forwa...,4
4,It was a decent read.. typical story line. Not...,3
...,...,...
9995,The whole series was great! Melody is a fanta...,5
9996,I didn't thing that much of this book. I am a...,3
9997,It is an emotional TRIP to the past with Trip ...,5
9998,This definitely got under my veins whereby I h...,5


In [1566]:
Books_small.shape

(10000, 10)

In [1567]:
Books_small['overall'].value_counts() #<---Gives Us Number Of Occurences Of A Particular Value in Digits

5    5930
4    2448
3     978
2     375
1     269
Name: overall, dtype: int64

In [1568]:
Books_small['overall'].value_counts(normalize=True)  #<---Gives Us Number Of Occurences Of A Particular Value in Fractions

5    0.5930
4    0.2448
3    0.0978
2    0.0375
1    0.0269
Name: overall, dtype: float64

In [1569]:
Books_small.sort_values(by='overall', ascending=False).head() #<--Starts From Big And Goes Down

,reviewerID,asin,reviewerName,helpful/0,helpful/1,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1F2H80A1ZNN1N,B00GDM3NQC,Connie Correll,0,0,"I bought both boxed sets, books 1-5. Really a...",5,Can't stop reading!,1390435200,"01 23, 2014"
5801,A1TZTFYEOPR4CO,1250037220,Brainchild,0,0,This was a great read. I really enjoyed this b...,5,Excellent read,1402617600,"06 13, 2014"
5797,AFE0AGQT50MBJ,1449417094,KV Trout,0,0,"There I said it. So call me classless, tastele...",5,I like Thomas Kinkade's style,1396828800,"04 7, 2014"
5795,A2H0KFB0IUNRT7,B005REXCKE,Stanley C. Pate,0,0,I have to admit that I purchased this book thi...,5,A very enjoyable read!,1393545600,"02 28, 2014"
5794,A2YJ8VP1SSHJ7,B007YN8IGE,NFerguson,0,0,The bound series has always intrigued me. I lo...,5,Bound By The Night,1391040000,"01 30, 2014"


In [1570]:
Books_small.sort_values(by='overall', ascending=True).head() #<--Starts From Low And Goes Up

,reviewerID,asin,reviewerName,helpful/0,helpful/1,reviewText,overall,summary,unixReviewTime,reviewTime
3419,AQY5XBYSENNZQ,1250006082,Quickbeam,3,3,"Based upon the blurb alone, I thought this sou...",1,A leaden Edwardian slog.,1400544000,"05 20, 2014"
8649,A2PWDBZQPR9JU6,0399161465,"Stacie Gorkow ""Sincerely Stacie""",3,3,Rose is a typist in the local police precinct....,1,Couldn't Finish It - Just Didn't Work for Me,1398297600,"04 24, 2014"
2730,A9FIEIKKPTX89,B00HOHMIVQ,"Steven M. Morris ""booklover""",6,7,"When is a couple doing the killing, She is alw...",1,Not very detailed!,1392076800,"02 11, 2014"
1595,AKJEYA96I1QE6,0764204319,nyccowgirl,1,2,There were a number of times when I wondered w...,1,Slow...,1390694400,"01 26, 2014"
2731,A3LUUNZVGSSA3M,0446517321,"Timmy God lover ""Tim E B""",0,1,This book depicts women as they really do not ...,1,Degrading,1400457600,"05 19, 2014"


In [1571]:
Books_small.describe()

,helpful/0,helpful/1,overall,unixReviewTime
count,10000.000000,10000.000000,10000.000000,1.000000e+04
mean,0.583800,0.797400,4.339500,1.396855e+09
std,2.156396,3.163219,0.983736,4.931959e+06
min,0.000000,0.000000,1.000000,1.388534e+09
25%,0.000000,0.000000,4.000000,1.392509e+09
50%,0.000000,0.000000,5.000000,1.396915e+09
75%,1.000000,1.000000,5.000000,1.401062e+09
max,100.000000,154.000000,5.000000,1.406074e+09


In [1572]:
Books_small.head()

,reviewerID,asin,reviewerName,helpful/0,helpful/1,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1F2H80A1ZNN1N,B00GDM3NQC,Connie Correll,0,0,"I bought both boxed sets, books 1-5. Really a...",5,Can't stop reading!,1390435200,"01 23, 2014"
1,AI3DRTKCSK4KX,B00A5MREAM,Grandma,0,0,I enjoyed this short book. But it was way way ...,3,A leaf on the wind of all hallows,1399593600,"05 9, 2014"
2,A3KAKFHY9DAC8A,0446547573,"toobusyreading ""Inspired Kathy""",1,1,I love Nicholas Sparks. I&#8217;ve read everyt...,4,Great writing from Nicholas Sparks.,1404518400,"07 5, 2014"
3,ATYBCYD6BIXVL,0955809215,Chrissie,0,0,I really enjoyed this adventure and look forwa...,4,great,1389225600,"01 9, 2014"
4,A17K95SEU3J68U,0991500776,"Sirde ""artist761""",0,0,It was a decent read.. typical story line. Not...,3,It was a decent read.. typical story line ...,1404864000,"07 9, 2014"


In [1573]:
Books_small.tail()

,reviewerID,asin,reviewerName,helpful/0,helpful/1,reviewText,overall,summary,unixReviewTime,reviewTime
9995,A32FV8MF6KTRYE,0615676863,S.Wilson,0,0,The whole series was great! Melody is a fanta...,5,Great,1389830400,"01 16, 2014"
9996,A2IYZ41783TX4R,B00GG2DWO2,"Lynn Demsky ""Lynn/MI""",0,0,I didn't thing that much of this book. I am a...,3,an anchor might have helped....,1391731200,"02 7, 2014"
9997,A1GHQOXZME7VLL,1482012294,CABorgford,0,0,It is an emotional TRIP to the past with Trip ...,5,Must Read Series - Timing is Everything,1388880000,"01 5, 2014"
9998,A37ZXVSFNM1UAZ,1481978063,"V. Jones ""Phoenix Rising""",0,0,This definitely got under my veins whereby I h...,5,This story grew on me!,1392076800,"02 11, 2014"
9999,A1EZD7IJOOAF6S,0956998569,Amazon Customer,1,1,Highly recommend this entire trilogy. It is ve...,4,truly enjoyed,1402358400,"06 10, 2014"


In [1574]:
Books_small.columns

Index(['reviewerID', 'asin', 'reviewerName', 'helpful/0', 'helpful/1',
       'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'],
      dtype='object')

In [1575]:
Books_small.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   reviewerID      10000 non-null  object
 1   asin            10000 non-null  object
 2   reviewerName    9957 non-null   object
 3   helpful/0       10000 non-null  int64 
 4   helpful/1       10000 non-null  int64 
 5   reviewText      10000 non-null  object
 6   overall         10000 non-null  int64 
 7   summary         10000 non-null  object
 8   unixReviewTime  10000 non-null  int64 
 9   reviewTime      10000 non-null  object
dtypes: int64(4), object(6)
memory usage: 781.4+ KB


In [1576]:
Books_small.index

RangeIndex(start=0, stop=10000, step=1)

In [1577]:
Books_small.isnull()

,reviewerID,asin,reviewerName,helpful/0,helpful/1,reviewText,overall,summary,unixReviewTime,reviewTime
0,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
9995,False,False,False,False,False,False,False,False,False,False
9996,False,False,False,False,False,False,False,False,False,False
9997,False,False,False,False,False,False,False,False,False,False
9998,False,False,False,False,False,False,False,False,False,False


In [1578]:
Sentiments=[]
for num in Books_small['overall']:
  if num>4.0:
    Sentiments.append("Positive")
  else:
    Sentiments.append("Negative")
print(len(Sentiments))

10000


In [1579]:
data={'Ratings':Books_small['overall'],'Sentiments':Sentiments,'Customer Reviews':Books_small['reviewText']}
Target_Data=pd.DataFrame(data)
Target_Data.sample(5)

,Ratings,Sentiments,Customer Reviews
4809,5,Positive,OMG what an awesome job by the author. Could ...
8744,5,Positive,Tightly constructed narrative moving readers f...
8977,5,Positive,I liked how this series turned out and I just ...
8928,5,Positive,I think Beauty and the Beast is a favourite of...
6974,5,Positive,This book is a great resource for anyone who i...


In [1580]:
training,test=train_test_split(Target_Data,test_size=0.33,random_state=42,stratify=Target_Data['Sentiments'])

In [1581]:
training.shape

(6700, 3)

In [1582]:
test.shape

(3300, 3)

In [1583]:
training

,Ratings,Sentiments,Customer Reviews
7205,1,Negative,I'm absolutely sick and tired of the man cheat...
6318,4,Negative,While Alice tries to &#8220;find herself&#8221...
6288,5,Positive,I found the stories contained in this set to b...
1042,5,Positive,Love the power these women portray and the hun...
4694,4,Negative,I like this book. It gives us a very healthy a...
...,...,...,...
7465,5,Positive,I started my online journey in 2009 and basica...
9250,4,Negative,My mom read this book as a child. It was her ...
2599,4,Negative,again Dan Gookin assistance is great for someo...
9597,5,Positive,This is a great read. The return to Broslin C...


In [1584]:
training['Customer Reviews'].iloc[3] #<---The Index Has Been Re-indexed When It Comes To iloc

'Love the power these women portray and the hunks that love them,'

In [1585]:
training['Sentiments'].iloc[3]

'Positive'

In [1586]:
training['Sentiments'].value_counts() # <---Data Is Biased

Positive    3973
Negative    2727
Name: Sentiments, dtype: int64

**BAG OF WORDS**

In [1587]:
train_x=training['Customer Reviews']
train_y=training['Sentiments']
test_x=test['Customer Reviews']
test_y=test['Sentiments']

In [1588]:
test_x.iloc[1]

"I really enjoyed this book. I couldn't put this book down until I finished it. The characters were good and the story line volume good."

In [1589]:
vectorizer = TfidfVectorizer()
train_xv = vectorizer.fit_transform(list(train_x))
test_xv=vectorizer.transform(list(test_x))

In [1590]:
train_xv.shape

(6700, 26663)

In [1591]:
test_xv.shape

(3300, 26663)

In [1592]:
train_y.value_counts()

Positive    3973
Negative    2727
Name: Sentiments, dtype: int64

**TESTING ON MULTIPLE CLASSIFICATION ALGORITHMS**

**LINEAR SVM**

In [1593]:
clf_svm=svm.SVC(kernel='linear')
clf_svm.fit(train_xv,train_y)
clf_svm.predict(test_xv[1])

array(['Positive'], dtype=object)

**DECISION TREE**

In [1594]:
clf_DecTree=DecisionTreeClassifier()
clf_DecTree.fit(train_xv,train_y)
clf_DecTree.predict(test_xv[1])

array(['Positive'], dtype=object)

**LOGISTIC REGRESSION**

In [1595]:
clf_log=LogisticRegression(max_iter=1000)
clf_log.fit(train_xv,train_y)
clf_log.predict(test_xv[1])

array(['Positive'], dtype=object)

**NAIVE BAYES**

In [1596]:
clf_gnb=GaussianNB()
clf_gnb.fit(train_xv.toarray(),train_y)
clf_gnb.predict(test_xv[1].toarray())

array(['Negative'], dtype='<U8')

**PERFORMANCE**

MEAN ACCURACY

In [1597]:
clf_svm.score(test_xv,test_y)

0.7306060606060606

In [1598]:
clf_DecTree.score(test_xv,test_y)

0.6115151515151516

In [1599]:
clf_gnb.score(test_xv.toarray(),test_y)

0.5187878787878788

In [1600]:
clf_log.score(test_xv,test_y)

0.733030303030303

F-1 SCORE

In [1601]:
f1_score(test_y,clf_svm.predict(test_xv),average=None)

array([0.64482621, 0.78301196])

In [1602]:
f1_score(test_y,clf_DecTree.predict(test_xv),average=None)

array([0.51476154, 0.67609904])

In [1603]:
f1_score(test_y,clf_log.predict(test_xv),average=None)

array([0.63519669, 0.78948626])

In [1604]:
f1_score(test_y,clf_gnb.predict(test_xv.toarray()),average=None)

array([0.51761847, 0.51995163])

In [1605]:
example=['do not buy this bad book', ' love this Book','impressive book']
example=vectorizer.transform(example)
clf_svm.predict(example)

array(['Negative', 'Positive', 'Positive'], dtype=object)

**GRID SEARCH**

In [ ]:
vector_machine=svm.SVC()
parameters={'kernel':('rbf','linear'),'C':(1,2,3,6,7)}
clf=GridSearchCV(vector_machine,parameters,cv=5)
clf.fit(train_xv,train_y)